In [1]:
import spacy



In [2]:
# Load pre-trained SpaCy model
nlp = spacy.load("en_core_web_sm")


In [6]:
import json
from spacy.tokens import DocBin

def convert_to_spacy_format(json_data):
    db = DocBin()
    for entry in json_data:
        text = entry["data"]["text"]
        annotations = [(ann["value"]["start"], ann["value"]["end"], ann["value"]["labels"][0]) for ann in entry["annotations"][0]["result"]]
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annotations:
            span = doc.char_span(start, end, label=label)
            if span:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return db

train_judgement_annotations = "./NER_TRAIN/NER_TRAIN_JUDGEMENT.json"
train_preamble_annotations = "./NER_TRAIN/NER_TRAIN_PREAMBLE.json"

dev_judgement_annotations = "./NER_DEV/NER_DEV_JUDGEMENT.json"
dev_preamble_annotations = "./NER_DEV/NER_DEV_PREAMBLE.json"

test_judgement_annotations = "./NER_TEST/NER_TEST_JUDGEMENT.json"
test_preamble_annotations = "./NER_TEST/NER_TEST_PREAMBLE.json"


In [8]:
#  "./spacy/train.spacy"

# Load JSON data and convert it
with open(train_judgement_annotations, "r") as j, open(train_preamble_annotations, "r") as p:
    
    json_data = json.load(j) + json.load(p)
db = convert_to_spacy_format(json_data)
db.to_disk("./spacy/train.spacy")


In [9]:
# "./spacy/dev.spacy"

with open(dev_judgement_annotations, "r") as j, open(dev_preamble_annotations, "r") as p:
    json_data = json.load(j) + json.load(p)
db = convert_to_spacy_format(json_data)
db.to_disk("./spacy/dev.spacy")

In [10]:
# "./spacy/test.spacy"

with open(test_judgement_annotations, "r") as j, open(test_preamble_annotations, "r") as p:
    json_data = json.load(j) + json.load(p)
db = convert_to_spacy_format(json_data)
db.to_disk("./spacy/test.spacy")

In [13]:
!python -m spacy init config config.cfg --pipeline ner --optimize efficiency

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


c:\Users\VICTUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\VICTUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [14]:
!python -m spacy train config.cfg --output ./output --paths.train ./spacy/train.spacy --paths.dev ./spacy/dev.spacy --paths.test ./spacy/test.spacy


✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     41.24    0.44    0.27    1.24    0.00
  0     200       4393.48   3667.58    9.12   15.67    6.44    0.09
  0     400       2052.88   2854.32   16.12   31.30   10.85    0.16
  0     600       4415.06   3276.06   25.85   32.43   21.49    0.26
  0     800       4660.20   3787.48   36.05   45.28   29.94    0.36
  0    1000       1604.12   2961.84   44.45   50.53   39.68    0.44
  0    1200       4770.89   3705.30   47.94   55.06   42.44    0.48
  0    1400       1654.87   3677.47   51.85   52.38   51.34    0

c:\Users\VICTUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\VICTUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:
import spacy
nlp = spacy.load("./output/model-best")

# Test with a new sentence
# text = "The Supreme Court heard the case under the IPC"
text = """o. 273 of 1951.
Appeal under articles 132 (1) and 134 (1)(c) of the Constitution of India from the Judgment and Order dated I3th October, 1950, of the High Court of Judicature at Patna (Shearer, Ramaswami and Sarjoo Prosad JJ.) in Miscellaneous Judicial Case No. 220 of 1949.
S.K. Mitra (K. Dayal, with him), for the appellant.
Basant Chandra Ghosh and Arun Chandra Mitra for the respondent.
May 26.
The Court delivered judgment as follows: MAHAJAN J.
This appeal has been preferred by the State of Bihar against the judgment of a Special Bench of the High Court of Judicature at Patna allowing the application of the respondent under section 23 of the Indian Press (Emergency Powers)Act, XXIII of 1931.
It appears that the petition was argued by both the sides as it was one made under article 926 of the Constitution.
The respondent was the keeper at all relevant times of the Bharati Press at Purulia, A pamphlet under 85 656 the heading "Sangram" was printed at the said press and is alleged to have been circulated in the town of Purulia in the district of Manbhum.
The Government of Bihar considered that the pamphlet contained objectionable matter of the nature described under section 4 (1) of the Indian Press (Emergency Powers) Act and required the press to furnish security in the sum of Rs. '2,000, under section 3(3) of the Act by the 19th September, 1949.
On the 26th September, 1949, the respondent applied to the High Court under section 23 for setting aside the above order.
This application was allowed by the majority of the Judges constituting the Bench.
Shearer J. was of the view that the application should be dismissed.
Several objections were raised to the validity of the order passed by the Bihar Government but it is unnecessary to mention all of them.
The two points which were seriously pressed before the High Court were that the leaflet did not contain any words or signs or visible representation of the nature described in section 4 (1) of the Act, and that the provisions of section 4 (1) of the Act were inconsistent with article 19 (1) of the Constitution and as such void under article 13.
The High Court reached the conclusion that the pamphlet did come within the mischief of the Act.
Sarjoo Prosad J., with whom Ramaswami J. concurred, on a construction of the decisions of this Court in Romesh Thapar vs The State of Madras(1), and Brij Bhushan V.
The State of Delhi(2), found, though with some reluctance, that section 4 (1) (a) of the Act was repugnant to the Constitution and therefore void.
Mr. Justice Shearer, however, held that the pamphlet was a seditious libel and that there was nothing in the two decisions of the Supreme Court referred to above which compelled the court to hold the provisions of section 4 (1) (a) of the Act to be void.
In my opinion, Shearer J. was right in the view that there is nothing in the two decisions of this Court which bears directly or indirectly on the point at issue in the present case and that both Sarjoo Prosad (1) [1950] S.C.R.594.
(2) ; 657 and Ramaswami JJ.
were in error in holding that these deci sions were conclusive on the question of the invalidity of clauses (a) and (b) of section 4 (1) of the Act.
Towards the concluding part of his judgment Sarjoo Prosad J. ob served as follows: "I am compelled to observe that from the above discus sions of the Supreme Court judgments, it follows logically that if a person were to go on inciting murder or other cognisable offences either through the press or by word of mouth, he would be free to do so with impunity inasmuch as he would claim the privilege of exercising his fundamental right of freedom of speech and expression.
Any legislation which seeks or would seek to curb this right of the person concerned would not be saved under article 19 (2) of the Constitution and would have to be declared void.
This would be so, because such speech or expression on the part of the individual would fall neither under libel nor slander nor defamation nor contempt of court nor any matter which of fends against decency or morality or which undermines the security of or tends to overthrow the State.
I cannot with equanimity contemplate such an anomalous situation but the conclusion appears to be unavoidable on the authority of the Supreme Court judgments with which we are bound.
I, there fore, wish that my decision on the point would sooner than ever come to be tested by the Supreme Court itself and the position reexamined in the light of the anomalous situation pointed out above.
It seems to me that the words used in the Constitution Act should be assigned a wide and liberal connotation even though they occur in a clause which pro vides an exception to the fundamental right vouchsafed under article 19 (1)(a) of the Constitution Act." These observations I speak with great respect disclose a complete lack of understanding of the precise scope of the two decisions of this Court referred to above.
Section 3 (3) of the Act under which the notice was issued in the present case enacts as follows: "Whenever it appears to the Provincial Government that any printing press is used for the purpose 658 printing or publishing any newspaper, book or other document containing any words, signs or visible representation of the nature described in section 4,sub section (1), the Provin cial Government may, by notice in writing to the keeper of the press . .order the keeper to deposit with the Magis trate security . " Clause (a) of section 4 (1) deals with words or signs or visible representations which incite to or encourage, or tend to incite to or encourage the commission of any offence of murder or any cognizable of fence involving violence.
It is plain that speeches or expressions on the part of an individual which incite to or encourage the commission of violent crimes, such as murder, cannot but be matters which would undermine the security of the State and come within the ambit of a law sanctioned by article.
19(2) of the Constitution.
I cannot help observing that the decisions of this Court in Romesh Thapar 's case(1), and in Brij Bhushan 's case(2) have been more than once misapplied and misunderstood and have been construed as laying down the wide proposition that restrictions of the nature imposed by section 4(1)(a) of the Indian Press (Emergency Powers) Act or of similar character are outside the scope of article 19(2) of the Constitution inasmuch as they are conceived generally in the interests of public order.
Sarjoo Prosad J. also seems to have fallen into the same error.
The question that arose in Romesh Thapar 's case(1) was whether the impugned Act (Madras Maintenance Public Order Act, XXIII of 1949) in so far as it purported by section 9 (1 A) to authorise the Provincial Government "for the purpose of securing the public safety and the mainte nance of public order, to prohibit or regulate the entry.into or the circulation, sale or distribution in the Province of Madras or any part thereof any document or class of documents" was a law relating to any matter which under mined the security of or tended to overthrow the State, and it was observed that whatever ends the impugned Act may have been intended to subserve and whatever (1)[1950] section
C.R. 594.
(2) [1950] S.C.R. 605.
659 aims its framers may have had in view, its application and scope could not, in the absence of delimiting words in the statute itself, be restricted to those aggravated forms of prejudicial activity which are calculated to endanger the security of the State, nor was there any guarantee that those authorized to exercise the powers under the Act would in using them discriminate between those who act prejudical ly to the security of the State and those who do not.
Sec tion 4(1)(a) of the impugned Act, however, is restricted to aggravated forms of prejudicial activity.
It deals specifi cally with incitement to violent crimes and does not deal with acts that generally concern themselves with the mainte nance of public order.
That being so, the decision in Romesh Thalbar 's case(1) given on the constitutionality of section 9(1 A) of the Madras Maintenance of Public Order Act has no relevancy for deciding the constitutionality of the provi sions of section 4(1)(a) of the Indian Press (Emergency Powers) Act.
Towards the concluding portion in Romesh Tha par 's judgment(1) it was observed as follows : "We are therefore of opinion that unless a law restrict ing freedom of speech and expression is directed solely against the undermining of the security of the State or the overthrow of it, such law cannot fall within the reservation under clause (2) of article although the restrictions which it seeks to impose may have been conceived generally in the interests of public order.
It follows that section 9(I A) which authorizes imposition of restrictions for the wider purpose of securing public safety or the maintenance of public order falls outside the scope of authorized restric tions under clause (2), and is therefore void and unconsti tutional.
" The restrictions imposed by section 4(1)(a) of the Indian Press (Emergency Powers) Act on freedom of speech and expression are solely directed against the undermining of the security of the State or the overthrow of it and are within the ambit of article 19(2) (1) 94. 660 of the Constitution.
The deduction that a person would be free to incite to murder or other cognizable offence through the press with impunity drawn from our decision in Romesh Thapar 's case(1) could easily have been avoided as it was avoided by Shearer J. who in very emphatic terms said as follows: " I have read and re read the judgments of the Supreme Court, and I can find nothing in them myself which bear directly on the point at issue, and leads me to think that, in their opinion, a restriction of this kind is no longer permissible.
" Be that as it may, the matter is now concluded by the language of the amended article 19(2) made by the Constitu tion (First Amendment) Act which is retrospective in opera tion, and the decision of the High Court on this point cannot be sustained.
Basant Chander Ghosh contended that the amendment made in article 19 (2) of the Constitution with retrospective operation was repugnant to article 20 of the Constitution inasmuch as it declared a certain act an offence which was not an offence at the time when the act was committed.
This contention is untenable.
The respondent is alleged to have violated the provisions of section 4(1)(a) of the Indian Press (Emergency Powers) Act which was a law in force in the year 1949 when the offending pamphlet was published.
She has not been convicted of any offence so far and is not being again convicted for the same by reason of the amend ment in article 19(2).
Article 20 has no application whatev er to the present case.
Article 19(2) empowers a legislature to make laws imposing reasonable restrictions on the funda mental rights conferred under article 19(1) of the Constitu tion.
It does not declare any acts which were not offences before as offences with retrospective effect.
Moreover, in the year 1949 the respondent was not possessed of any funda mental right which could be said to have been contravened by the amendment.
Though, as I have said above, the High Court is in error in the finding that the provisions of section ,4(1)(a) (1) [1950] S.C.R. 594, 661 of the Indian Press (Emergency Powers) Act are repugnant to the Constitution, its judgment has to be maintained as it is also in error in holding that the pamphlet in question fell within the mischief of section4 (1)(a) of the Indian Press (Emergency Powers) Act.
The document is written in high flown Bengali language and contains a good deal of demagogic claptrap with some pretence to poetic flourish.
It enunciates certain abstract propositions in somewhat involved language and it cannot be followed except with considerable effort.
The High Court held that the document offended against the provisions of section 4(1)(a) inasmuch as certain parts of it contemplate a bloody and violent revolution and that the central theme that runs through the whole gamut of the offending pamphlet is that the author is anxious to bring about a bloody revo lution and change completely the present order of things by causing a total annihilation of the persons and the policies of those who according to him are in the opposite camp.
Particular reference was made to the following passages in the writing which in the opinion of the learned Judges support that conclusion.
The first of these passages is in these words : "Oh thou foolish oppressor, you want to cause abject terror in me with your red eyes and full throated voice do that, I am not afraid . .
My pro test is against parochial national politics.
"Another passage reads thus : `` Death is my secret love; poison is my drink the flames of fire are my sweet breeze; the wailing of a hundred be reaved childless mothers is just a tune in my flute; the weeping of widows at their widowhood is just a rhythm of my song.
" The next passage referred to is in these terms : "I am the cremation ground.
I am the bloodthirsty goddess Kali who lives and moves about in the cremation ground.
Plague or famine is my great joy . .
I am thirsty, I want blood, I want revolution,.
662 I want faith in the struggle.
Tear, tear the chain of wrongs; Break thou the proud head of the oppressor.
" Reference was also made to a passage in which the writer desires that his cries should be heard by people far and near, that his call should be hearkened far far away across the hills, the jungles, across the rivers and rivulets and all those who hear should come forward to join the ranks in destroying the oppressor and in which he claims that he is the messenger of death, that his revolutionary song signals the door of each of the listeners and signals to them to come out if they have life, if they have health, if they have courage to come and dash to pieces those who commit oppression on the mother, and he says that with the blood of those followers let the revolution grow.
It winds up with an invocation to the readers in these terms : "If you are true, if you are the gift of God, if you are not a bastard, then come forward with a fearless heart to struggle against the oppressors ' improper conduct, oppres sion and injustice.
We should not tolerate wrongful oppres sion.
Oh, thou the people with the burning pain of thine heart burn the heart of the oppressive, high handed oppres sor.
Let all wrongs, all high handedness, all oppressions, all tyrannies be burnt in the flame.
" It seems to me that the learned Judges of the High Court took this writing too seriously.
It did not deserve that consideration.
It is some kind of patch up work, with no consistency or cohesion between its different parts.
Por tions of it are unmeaning nonsense and in other parts it talks of revolution in the abstract.
There is no appeal to anybody in particular or for any known or specific cause.
No mention is made of any specific kind of oppression or injus tice that is intended to be remedied.
The desire is.
to change the face of the earth by ending all oppression, tyranny and injustice.
Their is no evidence whatsoever for connecting this pamphlet with any agitation or movement at the time it was written in that locality.
I have read the writing several times and I think that Mr. Ghosh is 663 right when he says that the pamphlet contains merely empty slogans, carrying no particular meaning except some amount of figurative expression or language borrowed at random from various authors with a touch of poetic flourish about it.
Writings of this characters at the present moment and in the present background of our country neither excite nor have the tendency to excite any person from among the class which is likely to read a pamphlet of this nature.
They will necessarily be educated people.
Such writings leave their readers cold and nobody takes them seriously.
People laugh and scoff at such stuff as they have become too familiar with it and such writings have lost all sting.
Any non descript person who promises to change the order of things by bloody revolution and assumes the role of a new Messiah is merely the laughing stock of his readers and creates an adverse impression against himself, rather than succeed in stirring up any excitement in the minds of the readers.
Rhetoric of this kind might in conceivable circumstances inflame passions as, for example, if addressed to an excited mob, but if such exceptional circumstances exist it was for the State Government to establish the fact.
In the absence of any such proof we must assume that the pamphlet would be read by educated persons in the quietness of their homes or in other places where the atmosphere is normal.
I would therefore hold, in the words of my brother Bose in Bhagwati Charan Shukla vs Government of C.P. & Berar(1), that though the pamphlet in question uses extravagant language and there is in it the usual crude emotional appeal which is the stock in trade of the demagogue as well as a blundering and ineffective attempt to ape the poets, that is all, and there is nothing more in it.
The time is long past when writings of this kind can in normal circumstances excite people to commit crimes of violence or murder or tend to excite any body to commit acts of violence.
Again the language employed is full of mysticism and (I) I.L.R. 664 cannot be easily understood and it creates no impression of any kind on any person.
In order to determine whether a particular document falls within the ambit section 4(1), the writing has to be considered as a whole and in a fair and free and liberal spirit, not dwelling too much upon isolated passages or upon a strong word here and there, and an endeavour should be made to gather the general effect which the whole composi tion would have on the mind of the public.
Expressions which are the stock in trade of political demagogues and have no tendency to excite anybody, and exaggerations in language cannot lead to that result.
The learned Government Advocate placed reliance on the decision of Harries C.J. in Badri Narain vs Chief Secretary, Bihar Govermnent(2).
The learned Chief Justice therein held that in order to show that cer tain words fall under section 4 (1) (a) it is not necessary to show that the words tend to incite or to encourage the commission of a particular offence or offences and that it is sufficient if they tend to incite to or to encourage the commission of cognizable offences of violence in general.
In that case, a poem entitled "Labourers, the mainstay 'of the world" began by emphasising that labourers are the mainstay of the present world and then proceeded to describe their unfortunate and pitiful lot.
In a subsequent portion the author stated that though speechless today, when organized, the labourers will be as powerful as millions and this portion of the poem ended with these words: "Why are you helplessly tolerating the exploitation of your masters." The remaining lines were as follows: "Labourers, raise now the cry of revolution.
The heavens will tremble, the Universe will shake and the flames of revolution will burst forth from land and water.
You who have been the object of exploitation, now dance the fearful dance of destruction on this earth; truly, labourers, only total destruction will (2) A.I.R. 1941 Pat.132 665 create a new world order and that will bring happiness to the whole world.
" It is quite clear that here an appeal was made to la bourers inciting and encouraging them to commit acts of violence.
The words used certainly tended to achieve that result.
They were no empty slogans or abstract propositions.
It had one consistent and coherent purpose, i.e., to excite labourers and to bring them into action.
Any observation made about this writing can have no apt application for the determination of the present case.
The learned Chief Justice in the concluding part of the judgment very pertinently pointed out that a commonsense interpretation must be given to the document complained of, the question to be answered always being, what impression will the documents or words give to a man of ordinary commonsense.
My answer to this query in the present case is that the document read at first sight is not intelligible unless it is explained to that man of ordinary commonsense by a learned person and hence it can by itself create no impression of any kind on such a person.
After the writing is explained to such a man, he will merely laugh at it and throw it in the waste paper basket without taking it seriously.
He will refuse to believe that a person of this kind can create a new world order by appealing to a bloody revolution.
As I pointed out in my judgment in Harkrishan Singh vs Emperor(1), the use of such words as appear in this document creates no impression on the mind of any reasonable reader.
That case dealt with clause (d) of section 4 (1), but the principle underlying it also applies to the construction of writings which are alleged to fall under section 4 (1) (a).
I do not mean to suggest or to lay down as a general propo sition that some of the words used in the pamphlet in ques tion in the context of any other writing would not fall within the mischief of section 4 (1) (a).
Certain parts of the pamphlet, if read as isolated passages, may have the tendency to excite people to commit (I) A,I.R, 666 crimes of violence but that is not the effect if the pam phlet is read in its entirety.
The result is that I would dismiss the appeal but in the circumstances would make no order as to costs.
The State Government has succeeded in its contention that sec tion 4 (1) (a) of the Act is constitutional and that was the real ground on which it came to this Court.
PATANJALI SASTRI C.J.
I agree with the judgment just delivered by my learned brother Mahajan J. and have nothing to add.
MUKHERJEA J. I concur in the judgment delivered by my learned brother Mahajan J. and I would like to say a few words, regarding the publication itself which led to the demand of security by the Government under the provision of the Indian Press (Emergency) Act.
The point that requires consideration is, whether the words contained in the impugned publication are of the nature described in section 4 (1) (a) of the Act; or in other words whether they incite to or encourage or tend to incite to or to encourage the commission of any offence of murder or any cognizable offence involving violence.
It is well settled that to arrive at a decision on this point, the writing is to be looked at as a whole without laying stress on isolated passages or particular expressions used here and there, and that the court should take into consideration what effect the writing is likely to produce on the minds of the readers for whom the publication is intended.
Account should also be taken of the place, circumstances and occa sion of the publication, as a clear appreciation of the background in which the words are used is of very great assistance in enabling the court to view them in their proper perspective.
The leaflet in question is entitled "Sangram" or struggle.
It is written in high flown Bengali prose with a large mixture of poetic expressions borrowed at random from the writings of some well known 667 poets of Bengal.
The object of the writing as far as could be gathered from the document is to give a poetic or ideal istic picture of what is meant and connotated by "struggle"or revolution.
The aim and end of "struggle ", as stated in the leaflet, is to wipe outs, "oppression, injus tice or wrong" which is "pervading all over the world from the past to the future"; and it is only after all wrongs, injustice and oppression have perished that a new world could be built up.
This seems to be the main or central theme of the composition, clothed, though it is, under much incoherent talk and seemingly meaningless utterances.
There is no indication throughout the writing as to what kind of oppression, injustice or wrong the author had in mind.
Far from referring to grievances of any specific character, the writer does not even hint at such general causes of discon tent as political inequality, economic exploitation or class warfare which are the subject matter of agitation in many parts of the world.
The leaflet does not give indication also of any unpopular measure or act of injustice affecting the minds of the people in the particular area where it was published and within which it was intended to be circu lated.
In one part of the document the following words are found to occur: "If mother be true, let no disgrace spread in the name of the mother.
If mother tongue be equal to mother, then the said language is your most revered goddess.
Do not allow disgrace to spread in her name".
It is not the case of the Government and there is no statement or affidavit to that effect, that the passages here have any reference to the language controversy which agitated and probably is still agitating this particular district.
In another part of the document the expression "narrow parochial politics" has been used, but here again the Government has not made any attempt to explain, what this expression could, in the particular context, mean or refer to.
As no acts of injustice or oppression are actually mentioned in the document, it is difficult to say who the "oppressors" are, whose "proud heads" the author asks his 668 readers to break.
It is quite clear that the "oppressor" mentioned here is neither the Government nor the party in power, nor has it any relation to any particular class of persons or a sect or community which might be harassing others and trampling upon their rights.
It may be, that to attract the operation of section 4 (1) (a) of the Indian Press Act, the incitement to murder or violence need not be specifically directed against particular individuals or class of persons; but when the whole talk is about injustice or oppression in the abstract, which is stated by the author to be in existence from the beginning of time and when in hyperbolic language a hope is expressed of establishing a better and a cleaner world through struggle, sweat and blood, the words used may not improperly be looked upon as an effusion of poetic fancy which, having no relation to actual facts can have very little potency for doing mis chief.
I will now proceed to examine the contents of the pamphlet in detail.
The writer begins in an affected poetic vein and de scribes, in language, to which it is difficult to attach any rational meaning, what "struggle" or revolution is.
The "struggle" which is personified in the article introduces itself in the following manner: "I am not wealth, nor popular strength, not the people nor fame;.
I am not joy nor a brag, nor the timid look of the beloved 's eyes .
I am not mother 's affection, nor sister 's love".
If these words convey any sense, they can only mean that the struggle or revolution which the writer wants to depict is something different from what we ordinarily associate with our social life and happiness; it is a negation of all natural human feelings and sentiments.
The next paragraph says in equal enigmatical language what "Sangram" or "strug gle" actually is.
"I am old antiquated history" thus the article proceeds; " I am time eternal, I am the future, the present and the past, in my heart is written the story of the past, the problems of the present and the voice of the 669 future".
I do not know whether this is a poetic way of depicting the entire life process which is said to lie through struggle and guide our evolution in this planet.
Struggle, according to the author, is coeval with time and eternity.
In the next paragraph the writer passes on to say with many repetitions of the word "wrong" that "it is wrong which is pervading all over from the past to the future", and it is this wrong that is to be righted by the struggle.
The struggle here is likened for reasons best known to the author to a piece of torn grass in the middle stream of a turbulent river, and to a grain of dust thrown in the face of a cyclone.
"It is dishonour, Unhappiness, endless pain.
" It is again likened successively to the frown of the be loved, to famine, storm and evil days.
The call is sent to everybody to come on "where the sky is cracking and the endless rough and thorny path is shrouded in darkness" and assist in building up a new world.
Many of the expressions used here are taken verbatim from the writings of some well known Bengalee authors, though they sound nothing but a rigmarole in the present context.
The next paragraph begins with the word "revolution".
Struggle is revolution and through struggle and revolution the world is to be built anew.
It is then said that "death is my darling and death is the only truth in this world".
If one has to die, there is no sense in dying of illness.
Let a man choose an honourable death by standing against oppressors.
Quite abruptly the author brings in the name of Sri Subhas Chandra Bose in the midst of this talk and asks his readers to listen "far far away across the hills, across the jungle, across the rivers and rivulets the call of Subhas Chandra Bose, the greatest revolutionary leader of the world".
The people are asked not to stop until the objective is attained.
Again it is said "I am struggle, I am revolution .
I am a Hindu, I am a Mussalman, I am a Christian, I am a Jew, I am a Keduin, I am severed from all religions by the fruits of my action in previous births".
Without the least attention to any sequence of thought, immediately 670 after this, the imaginary oppressor is addressed by the author as follows: "Oh you foolish oppressor you want to terrify with your red eyes, I fear not." The author, or rather the personified "struggle" which purports to speak, then repeats the well known words of poet Tagore and says that he does not seek salvation through renunciation; he wants that salvation which lies in joy amidst innumerable dangers and difficulties.
The idea of finding joy in all that is hated, avoided and dreaded in this world is elaborated in the passages that follow. "Death" it is said "is my secret love, poison is my drink, the flames of fire are my sweet breeze, the cry of childless mothers a tune in my flute and the weeping of widows a rythm of my song".
In this vein the author goes on conjuring up all the uncanny and weird things in the world and associat ing them with struggle.
"I am not joy, I am the remnant of the dying cries .
I am the bloodthirsty goddess Kali who lives and moves about in the cremation ground.
I want blood . .
Break the proud head of the oppressor.
I bathe in flames . . .
Thunder is my kiss of affection . .
I do not understand myself.
I do not know myself.
I do not recognise myself still I want revo lution, still I want struggle".
The learned Judges of the High Court laid very great stress on these passages which in their opinion constitute a direct incitement to bloody revolution; and that is also the line of argument adopted by Mr. Mitter who appeared before us on behalf of the State.
It has been argued by Mr. Ghosh appearing for the respondent that the "struggle" which the author has depicted and which he aims at is a non violent struggle and the blood that is to be shed is the blood of those who are called upon to resist oppression and injustice.
On the other hand, it is argued on behalf of the State that the passages quoted above can only mean that it is a bloody and violent revolution which could carry men to their desired end.
In my opinion, neither of these contentions furnish to us the proper method of approach to the question which requires 671 decision in the present case.
We would have to look at the article as a whole and focus our attention on what can be regarded to be its central theme or purpose.
As has been said already, what the writer wants is to draw an ideal picture of "struggle" or revolution quite unconnected with any particular place, or any particular political or social environment.
Injustice or oppression exists, according to the author, from the very dawn of time and so also does struggle or revolution.
It is an integral part of the world process and is a sort of irrational or blind impulse.
This is expressed by saying "I do not understand myself,I do not recognise myself, still I want revolution".
In painting death or war, the artist would naturally choose some uncanny associations.
The trappings of revolution, as the author paints it, are all the fearful and hideous things in this world.
It is linked up with thunder and storm, fire and devastation, cataclysm, famine, danger, destruction and deat"""
doc = nlp(text)
dictionary = {}
for ent in doc.ents:
    if ent.label_ not in dictionary:
        dictionary[ent.label_] = [ent.text]
    else:
        dictionary[ent.label_].append(ent.text)

dictionary['PROVISION']
dictionary['STATUTE']

['articles 132 (1) and 134 (1)(c)',
 'section 23',
 'article 926',
 'section 4 (1)',
 'section 3(3)',
 'section 23',
 'section 4 (1)',
 'section 4 (1)',
 'article 19 (1)',
 'article 13',
 'section 4 (1) (a)',
 'section 4 (1) (a)',
 'clauses (a) and (b) of section 4 (1)',
 'article 19 (2)',
 'article 19 (1)(a)',
 'section 4,sub section (1)',
 'section 4 (1)',
 'section 4(1)(a)',
 'article 19(2)',
 'section 9 (1 A)',
 'section 9(1 A)',
 'section 4(1)(a)',
 'clause (2) of',
 'section 9(I A)',
 'clause (2)',
 'section 4(1)(a)',
 'article 19(2) (1) 94. 660',
 'article 19(2)',
 'article 19 (2)',
 'article 20',
 'section 4(1)(a)',
 'article 19(2)',
 'article 19(1)',
 'section ,4(1)(a) (1) [1950] S.C.R. 594, 661',
 'section 4(1)(a)',
 'section 4(1)',
 'section 4 (1) (a)',
 'clause (d) of section 4 (1)',
 'section 4 (1) (a)',
 'section 4 (1) (a)',
 'section 4 (1) (a)',
 'section 4 (1) (a)']